In [ ]:
import pandas as pd
import flywheel

In [ ]:
client = flywheel.Client()

Let's test moving a subject with the SDK

In [ ]:
test_session = client.get('5d4dac1fc3758a0041d6472d')
test_subject = client.get_subject(test_session.subject.id)
test_project = client.get(test_session.parents.project)

In [ ]:
print(test_project.label, test_subject.label, test_session.label)

In [ ]:
target_project = client.get('5c743dc7ba258000383968cd')
print(target_project.label)

In [ ]:
test_subject.update({'project': target_project.id})

Check changes:

In [ ]:
test_subject = test_subject.reload()

In [ ]:
client.get(test_subject.project).label

Fantastic!

Now return it:

In [ ]:
test_subject.update({'project': test_project.id})

In [ ]:
test_subject = test_subject.reload()
client.get(test_subject.project).label

---

Now we can work on moving the remaining PNC subjects.

In [ ]:
original = client.projects.find_first('label=ReproBrainChart')

In [ ]:
original_subjects = original.subjects()

In [ ]:
pnc_df = pd.read_csv('/storage/ttapera/RBC/data/bblids_3.csv', names=['bblid', 'hash', 'fullid', 'process'])

In [ ]:
pnc_df

In [ ]:
target_project = client.projects.find_first('label=RBC_PNC')
target_subjects = target_project.subjects()
target_sessions = client.get_project_sessions(target_project.id)

In [ ]:
len(target_sessions)

It looks like something went wrong with moving these subjects. For those that don't have data in `original` we'll have to go and get their data from the PNC.

In [ ]:
original_labels = [x.label for x in original_subjects]
target_labels = [x.label for x in target_subjects]

In [ ]:
for i, row in pnc_df.iterrows():
    
    # first check if the subject is in the original project
    print(row['bblid'], "({})".format(row['fullid']))
    in_original = row['fullid'] in original_labels
    print('In original:', in_original)
    
    
    # then check if the subject is in the target project
    in_target = row['fullid'] in target_labels
    print('In target:', in_target)
    
    # check if the subject has a session in target
    if in_target:
        t_subject = [x for x in target_subjects if x.label == row['fullid']][0]
        has_session = len(t_subject.sessions()) > 0
        print('Has session:', has_session)
    
    print()

In [ ]:
verbose = False
to_process = pd.DataFrame(columns=pnc_df.columns)

for i, row in pnc_df.iterrows():
    
    # first check if the subject is in the original project
    if verbose: print(row['bblid'], "({})".format(row['fullid']))
    in_original = row['fullid'] in original_labels
    if verbose: print('In original:', in_original)
    
    
    # then check if the subject is in the target project
    in_target = row['fullid'] in target_labels
    if verbose: print('In target:', in_target)
    if not in_target:
        to_process = to_process.append(row)
    
    # check if the subject has a session in target
    if in_target:
        t_subject = [x for x in target_subjects if x.label == row['fullid']][0]
        has_session = len(t_subject.sessions()) > 0
        if verbose: print('Has session:', has_session)
        if not has_session:
            to_process = to_process.append(row)
        
    if verbose: print()

In [ ]:
to_process

We also need to add the remaining 200 who had export errors:

In [ ]:
remaining='''
100031
100088
100115
100278
100401
100761
101098
101158
101327
101754
102467
102598
102954
103009
103541
103621
104207
104991
105083
105215
105309
105634
105672
105765
106919
107394
107522
107867
107903
108012
108034
108358
108372
108805
108952
109083
109301
109575
109600
109647
109649
109667
109731
109732
109853
109914
109980
110166
110176
110511
110586
110828
110961
111820
111873
111884
112269
112408
112624
112861
113433
113551
113744
113929
114069
114117
114165
114326
114436
114738
114785
114824
114891
114935
115051
115267
116130
116191
116371
116672
117529
117595
118615
118702
118708
119287
119386
120052
120508
120850
121476
122579
122801
122896
124947
125106
125250
125292
125505
125524
126230
126347
126425
126532
126837
126852
126960
127488
127527
127611
127731
127854
127977
128061
128079
128154
128208
128601
128691
128979
129272
129435
129486
129591
129741
130332
130438
130480
130639
130693
130759
130858
130908
131010
131076
132560
132791
132995
136004
138875
138929
139151
139427
139469
139490
80179
80199
81231
81353
81533
81662
82587
82709
82877
83013
83035
83113
83260
83322
83429
83454
84096
84814
85083
85412
87166
87282
87445
87563
87699
87905
88166
88209
88395
88589
88760
89300
89320
89502
89505
89557
90066
90736
90742
90849
91145
91466
91555
91674
92500
93020
93242
93282
93755
93891
94324
94572
94621
94643
94815
94906
95051
95387
95587
95754
95992
96201
96574
96591
96904
97124
97271
97485
98319
98320
98339
98425
98488
98555
98585
98934
99268
99480
99604
99648
'''

remaining = remaining.split()

In [ ]:
remaining

In [ ]:
verbose = True
to_process = pd.DataFrame(columns=pnc_df.columns)

for i, row in pnc_df.iterrows():
    
    # first check if the subject is in the original project
    if verbose: print(row['bblid'], "({})".format(row['fullid']))
    in_original = row['fullid'] in original_labels
    if verbose: print('In original:', in_original)
    
    
    # then check if the subject is in the target project
    in_target = row['fullid'] in target_labels
    if verbose: print('In target:', in_target)
    if not in_target:
        to_process = to_process.append(row)
        if verbose: print()
        continue
    
    # check if the subject has a session in target
    if in_target:
        t_subject = [x for x in target_subjects if x.label == row['fullid']][0]
        has_session = len(t_subject.sessions()) > 0
        if verbose: print('Has session:', has_session)
        if not has_session:
            to_process = to_process.append(row)
            if verbose: print()
            continue
    
    # check if subject is in remaining
    if str(row['bblid']) in remaining:
        if verbose: print("subject still hasn't been refaced")
        to_process = to_process.append(row)
        if verbose: print()
        continue

    if verbose: print()

In [ ]:
to_process

In [ ]:
to_process.to_csv('../data/reupload.csv', index=False)

In [ ]:
ls